In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from statsmodels.tsa.arima.model import ARIMA

# ---------- Config ----------
TICKERS = ["AAPL", "MSFT", "AMZN"]  # any tickers you want
START_DATE = (datetime.today() - timedelta(days=365*4)).strftime("%Y-%m-%d")

# ---------- Download price data ----------
def get_price_data(ticker):
    """
    Download adjusted daily close prices for the given ticker since START_DATE.
    Uses yfinance auto_adjust=True, so prices are already adjusted for splits/dividends.
    """
    df = yf.download(
        ticker,
        start=START_DATE,
        interval="1d",
        progress=False,
        auto_adjust=True
    )
    if "Close" not in df.columns or df.empty:
        raise ValueError(f"No 'Close' data for {ticker}.")
    return df[["Close"]].rename(columns={"Close": ticker})

# ---------- Risk metrics (absolute, no benchmark) ----------
def compute_risk_metrics(price_df: pd.DataFrame, ticker: str):
    """
    Returns annualized return, annualized volatility, Sharpe, and cumulative return.
    Sharpe uses a 0% risk-free rate for simplicity.
    """
    s = price_df[ticker].dropna()
    rets = s.pct_change().dropna()
    if rets.empty:
        return {
            "Ticker": ticker,
            "Cumulative Return": None,
            "Annualized Return": None,
            "Annualized Volatility": None,
            "Sharpe Ratio": None
        }

    # Returns
    cumulative_return = float(s.iloc[-1] / s.iloc[0] - 1.0)

    # Annualized metrics
    ann_return = float(rets.mean() * 252)
    ann_vol = float(rets.std(ddof=1) * np.sqrt(252))
    sharpe = ann_return / ann_vol if ann_vol != 0 else np.nan

    return {
        "Ticker": ticker,
        "Cumulative Return": round(cumulative_return, 4),
        "Annualized Return": round(ann_return, 4),
        "Annualized Volatility": round(ann_vol, 4),
        "Sharpe Ratio": None if np.isnan(sharpe) else round(sharpe, 2),
    }

# ---------- ARIMA forecast (simple) ----------
def forecast_price_arima(price_series: pd.Series, periods: int = 30):
    """
    Fit ARIMA(5,1,0) and forecast 'periods' trading days ahead.
    Returns the last forecasted price.
    """
    series = price_series.dropna().squeeze()
    if len(series) < 60:
        return np.nan  # not enough data for a stable fit
    try:
        model = ARIMA(series, order=(5, 1, 0))
        model_fit = model.fit()
        fc = model_fit.forecast(steps=periods)
        return float(fc.iloc[-1])
    except Exception:
        return np.nan

# ---------- Run ----------
rows = []
for ticker in TICKERS:
    px = get_price_data(ticker)

    # Risk metrics (absolute)
    risk = compute_risk_metrics(px, ticker)

    # ARIMA forecast (30 trading days)
    pred_price = forecast_price_arima(px[ticker], periods=30)
    risk.update({
        "Predicted Price (30d)": round(pred_price, 2) if pd.notna(pred_price) else None
    })

    rows.append(risk)

df_results = pd.DataFrame(rows)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")
display(df_results)

,Ticker,Cumulative Return,Annualized Return,Annualized Volatility,Sharpe Ratio,Predicted Price (30d)
0,AAPL,0.54,0.15,0.29,0.52,218.87
1,MSFT,0.87,0.19,0.27,0.72,521.36
2,AMZN,0.34,0.14,0.37,0.38,222.73
